In [2]:
from typing import List, Dict, Tuple, Optional
from collections import deque
import heapq

In [ ]:
# 20. Valid Parentheses

class Solution:
    def isValid(self, s: str) -> bool:
        paren = {'(':')', '{':'}', '[':']'}
        store = []
        for p in s:
            if p in paren:
                store.append(p)
            else:
                if not store or p != paren[store.pop()]:
                    return False
        return not store

s = "(())"
a = Solution()
res = a.isValid(s)
res

True

In [21]:
# 48. Rotate Image
class Solution:
    def rotate(self, matrix: List[List[int]]) -> None:
        """
        Do not return anything, modify matrix in-place instead.
        """ 
        l = len(matrix)
        # 1/ 第一层，确定几个环
        for i in range(l//2):
            top, left = i,i
            bottom, right = l-i-1, l-i-1
            # 2/ 按边交换元素
            for j in range(left, right):
                matrix[top][j], matrix[j][right], matrix[bottom][right+left-j], matrix[bottom+top-j][left]  = \
                    matrix[bottom+top-j][left], matrix[top][j], matrix[j][right], matrix[bottom][right+left-j]

matrix = [[5,1,9,11],[2,4,8,10],[13,3,6,7],[15,14,12,16]]
a = Solution()
res = a.rotate(matrix)
matrix

[[15, 13, 2, 5], [14, 3, 4, 1], [12, 6, 8, 9], [16, 7, 10, 11]]

In [28]:
# 54. Spiral Matrix
class Solution:
    def spiralOrder(self, matrix: List[List[int]]) -> List[int]:
        # 非正方形matrix都用while-loop进行边界压缩
        res = []
        top, left = 0, 0
        bottom, right = len(matrix)-1, len(matrix[0])-1
        while top <= bottom and left <= right:
            for i in range(left, right+1):
                res.append(matrix[top][i])
            top += 1
            for i in range(top, bottom+1):
                res.append(matrix[i][right])
            right -= 1

            # /边界值有变动，需要提前check
            if top <= bottom:
                for i in range(right, left-1, -1):
                    res.append(matrix[bottom][i])
                bottom -= 1

            if left <= right:
                for i in range(bottom, top-1, -1):
                    res.append(matrix[i][left])
                left += 1
        return res

matrix = [[1,2,3,4],[5,6,7,8],[9,10,11,12]]
a = Solution()
res = a.spiralOrder(matrix)
res

[1, 2, 3, 4, 8, 12, 11, 10, 9, 5, 6, 7]

In [ ]:
# 73. Set Matrix Zeroes
# 重点在于是否能够不随m和n增长
# edge flag。边缘单独处理，内部靠行列flag
class Solution:
    def setZeroes(self, matrix: List[List[int]]) -> None:
        """
        Do not return anything, modify matrix in-place instead.
        """
        # 1/ check
        first_row_zero, first_col_zero = False, False
        for i in range(len(matrix[0])):
            if matrix[0][i] == 0:
                first_row_zero = True
        for i in range(len(matrix)):
            if matrix[i][0] == 0:
                first_col_zero = True
        
        # 2/ flag
        for i in range(1, len(matrix)):
            for j in range(1, len(matrix[0])):
                if matrix[i][j] == 0:
                    matrix[i][0] = 0
                    matrix[0][j] = 0
        
        # 3/ convert
        for i in range(1, len(matrix[0])):
            if matrix[0][i] == 0:
                for j in range(len(matrix)):
                    matrix[j][i] = 0
        for i in range(1, len(matrix)):
            if matrix[i][0] == 0:
                for j in range(len(matrix[0])):
                    matrix[i][j] = 0
        
        # 4/ edge check
        if first_col_zero:
            for i in range(len(matrix)):
                matrix[i][0] = 0
        if first_row_zero:
            for i in range(len(matrix[0])):
                matrix[0][i] = 0

matrix = [[0,1,2,0],[3,4,5,2],[1,3,1,5]]
a = Solution()
res = a.setZeroes(matrix)
matrix

[[0, 0, 0, 0], [0, 4, 5, 0], [0, 3, 1, 0]]

In [25]:
# 84. Largest Rectangle in Histogram
'''
用 monotonic stack 存下标。
每根柱子只负责“自己当最低柱”的矩形,不会抢别人的宽度。
它是在保证：每根柱子，恰好一次，在“右边界第一次确定时”，把“以自己为最矮”的矩形算完。
真实的计算区间是[L+1 , i-1]
'''
class Solution:
    def largestRectangleArea(self, heights: List[int]) -> int:
        heights.append(0) # 哨兵
        area = 0
        stack = []
        for i in range(len(heights)):
            while stack and heights[i] < heights[stack[-1]]:
                idx = stack.pop()
                l = stack[-1] if stack else -1
                area = max(area, (i-l-1) * heights[idx])
            # 单调栈
            stack.append(i)
        return area

heights = [2,1,5,6,2,3]
a = Solution()
res = a.largestRectangleArea(heights)
res

10

In [2]:
# 155. Min Stack
# 这种组合题，就是要弄清楚，所需的数据结构最basic的是什么样的，每一个function核心功能是什么
# 在 push/pop 的同时，维护“到当前为止的最小值”。
# 本质上就是两个栈，一个正常出入栈、一个递减栈（取最小值），碰到最小值要变的时候才更新递减栈，本质上是记录历史上出现过的最小值
# 既然栈具有时间性（进入顺序），相应配合一个具有历史性的最小值记录，也是沿时间线进行关联

class MinStack:

    def __init__(self):
        self.stack = []
        self.minstack = []

    def push(self, val: int) -> None:
        self.stack.append(val)
        if not self.minstack:
            self.minstack.append(val)
        elif val <= self.minstack[-1]:
            self.minstack.append(val)

    def pop(self) -> None:
        out = self.stack.pop()
        if out == self.minstack[-1]:
            self.minstack.pop()

    def top(self) -> int:
        return self.stack[-1]

    def getMin(self) -> int:
        return self.minstack[-1]

# Your MinStack object will be instantiated and called as such:
obj = MinStack()
obj.push(2)
obj.push(4)
obj.pop()
param_3 = obj.top()
param_4 = obj.getMin()

In [ ]:
# 215. Kth Largest Element in an Array
'''
- 这道题是要找出数组中第 K 大的元素。输入是一个整数数组 nums 和一个整数 k，输出是数组中第 K 大的数字。
- 使用最简单的解法，可以先对数组进行降序排序，然后取第 K 个元素。时间复杂度是 O(n log n)。
- 我会使用一个优化解法：
    - 使用一个大小为 K 的最小堆，遍历数组，将每个元素加入堆中。
    - 如果堆的大小超过 K，就弹出堆顶（也就是当前最小值）。这样堆中始终保留的是当前最大的 K 个元素，堆顶就是第 K 大的数。
    - 这样做的时间复杂度是 O(n log k)，更高效。
- 总结一下这道题的思路：使用一个大小为 K 的小顶堆维护当前最大的 K 个数，遍历数组过程中更新堆的内容，最终堆顶就是我们要找的第 K 大的元素。
'''
class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        import heapq
        heap = []
        for n in nums:
            heapq.heappush(heap, n)
            if len(heap) > k:
                heapq.heappop(heap)
        return heap[0]
            
nums = [3,2,1,5,6,4]; k = 2
a = Solution()
res = a.findKthLargest(nums, k)
res

5

In [ ]:
# 240. Search a 2D Matrix II
# 过于优雅了，这个 m+n 的解法，没做过确实很难想到otz
class Solution:
    def searchMatrix(self, matrix: List[List[int]], target: int) -> bool:
        i,j = 0, len(matrix[0])-1
        while i < len(matrix) and j >= 0:
            if matrix[i][j] == target:
                return True
            elif matrix[i][j] < target:
                i += 1
            else:
                j -= 1
        return False

matrix =[[1,4,7,11,15],[2,5,8,12,19],[3,6,9,16,22],[10,13,14,17,24],[18,21,23,26,30]]
a = Solution()
res = a.searchMatrix(matrix, target=20)
res

False

In [2]:
# 295. Find Median from Data Stream
import heapq

class MedianFinder:
    def __init__(self):
        self.left = []
        self.right = []
    
    def addNum(self, num: int) -> None:
        # 1/ push in
        if not self.left or num <= -self.left[0]:
            heapq.heappush(self.left, -num)
        else:
            heapq.heappush(self.right, num)
        # 2/ rebalance if needed
        if len(self.left) > len(self.right) + 1:
            bottom = heapq.heappop(self.left)
            heapq.heappush(self.right, -bottom)            
        elif len(self.right) > len(self.left) :
            top = heapq.heappop(self.right)
            heapq.heappush(self.left, -top)

    def findMedian(self) -> float:
        if len(self.left) == len(self.right):
            return (abs(self.left[0])+self.right[0])/2
        else:
            return abs(self.left[0])

# Your MedianFinder object will be instantiated and called as such:
# obj = MedianFinder()
# obj.addNum(num)
# param_2 = obj.findMedian()

In [ ]:
# 347. Top K Frequent Elements
'''
- 这道题是要找出数组中出现频率最高的前 K 个元素。输入是一个整数数组 nums 和一个整数 k，输出是一个包含 k 个元素的列表，这些元素是数组中最常出现的元素。
- 使用最简单的解法，可以先统计每个元素出现的次数，然后将所有元素按频率排序，最后取前 K 个。这样做的时间复杂度是 O(n log n)，其中 n 是数组长度。
- 我会使用一个优化解法：
    - 先用 Counter 统计每个元素出现的频率，然后用一个最小堆（小顶堆）来维护前 K 个高频元素。
    - 每次插入新元素时，如果堆大小超过 K，就弹出堆顶（频率最小的）。最终堆中剩下的就是频率前 K 高的元素。
    - 这样做的时间复杂度是 O(n log k)，适合用于数据量大的场景。
- 总结一下这道题的思路：首先统计频率，然后利用小顶堆动态维护出现频率最高的 K 个元素，最后从堆中提取结果。
'''
class Solution:
    def topKFrequent(self, nums: List[int], k: int) -> List[int]:
        import heapq
        from collections import Counter
        count = Counter(nums)
        heap = []
        for num,freq in count.items():
            heapq.heappush(heap, (freq, num))
            if len(heap) > k:
                heapq.heappop(heap)
        return [ num for _, num in heap]

nums = [1,1,1,2,2,3]; k = 2
a = Solution()
res = a.topKFrequent(nums, k)
res

[2, 1]

In [ ]:
# 394. Decode String
'''
按部就班地弹出，本质上和括号是一样的
要注意：
- 字符串的顺序、单个字符串与已组合字符串的收集
- digit的顺序和数量（12->是十二不是一和二）
'''
class Solution:
    def decodeString(self, s: str) -> str:
        stack = []
        for i in range(len(s)):
            if stack and s[i] == ']':

                # alphabet
                part = []
                while stack[-1] != '[':
                    part.append(stack.pop())

                # 去'['
                stack.pop()
                
                # numeric
                num = ''
                while stack and stack[-1].isdigit():
                    num += stack[-1]
                    stack.pop()
                num = int(num[::-1])
                
                # add
                cur = ''.join(reversed(part))
                stack.append(num*cur)
                continue

            stack.append(s[i])
        return ''.join(stack)

s = "3[z]2[2[y]pq4[2[jk]e1[f]]]ef"
a = Solution()
res = a.decodeString(s)
res

'zzzyypqjkjkefjkjkefjkjkefjkjkefyypqjkjkefjkjkefjkjkefjkjkefef'

In [13]:
# 739. Daily Temperatures
'''
涉及list局部段长的，存 idx，j - i 是最合适作这种记录的;
要分清，原数据temparature的值和下标，存idx的stack的值和下标，会有嵌套调用的情况，比如temparatures[stack[0]]
复习一下占位res，比如[0]*1，或者[0]*26 - 涉及alphabet的都这样。如果返回的内容与输入是一一对应的，就可以考虑这种占位。

单调栈的铁律：谁在栈里，就必须是“已经出现但还没被结算的人”，所以不能用i和i-1，时空逻辑上应该是i和i+1，你只能存i而不是i-1

每天 i 到来时，顺序必须是：
- 用 temperatures[i] 去结算栈里的人
- 然后把 今天 i 自己 压栈
- 栈里永远只存“过去的人”
'''
class Solution:
    def dailyTemperatures(self, temperatures: List[int]) -> List[int]:
        res = [0] * len(temperatures)
        stack = []
        for i in range(len(temperatures)):
            while stack and temperatures[i] > temperatures[stack[-1]]:
                idx = stack.pop()
                res[idx] = i - idx
            stack.append(i)
        return res

temperature = [73,74,75,71,69,72,76,73]
a = Solution()
res = a.dailyTemperatures(temperature)
res

[1, 1, 4, 2, 1, 1, 0, 0]